In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

uri = 'duckdb:///../rentradar/db/rentradar.db'

## Setup LangChain

In [2]:
db = SQLDatabase.from_uri(
    uri,
    sample_rows_in_table_info=3
)

print(db.table_info)


CREATE TABLE counties (
	id VARCHAR, 
	county VARCHAR
)

/*
3 rows from counties table:
id	county
cd1120ac-f345-5b53-baab-74449599067b	Albemarle County
52c07453-ee28-5c0d-978d-d403c78b58af	Charlottesville City
84328055-b1cf-5279-a09a-e98f63e3d09b	Louisa County
*/


CREATE TABLE current_market_stats (
	bedrooms INTEGER, 
	"averageRent" INTEGER, 
	"minRent" INTEGER, 
	"maxRent" INTEGER, 
	"totalListings" INTEGER, 
	"lastUpdatedDate" VARCHAR, 
	"zipCode" INTEGER
)

/*
3 rows from current_market_stats table:
bedrooms	averageRent	minRent	maxRent	totalListings	lastUpdatedDate	zipCode
1	1439	610	3000	35	2024-02-24T00:00:00.000Z	22903
2	1734	875	2850	60	2024-02-24T00:00:00.000Z	22903
3	2260	1425	4000	46	2024-02-24T00:00:00.000Z	22903
*/


CREATE TABLE historic_market_stats (
	bedrooms INTEGER, 
	"averageRent" REAL, 
	"minRent" INTEGER, 
	"maxRent" INTEGER, 
	"totalListings" INTEGER, 
	date VARCHAR, 
	"zipCode" INTEGER
)

/*
3 rows from historic_market_stats table:
bedrooms	averageRent	minRent

/home/timboslice/.cache/pypoetry/virtualenvs/rentradar-ZiKOXBpo-py3.10/lib/python3.10/site-packages/duckdb_engine/__init__.py:181: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [3]:
toolkit = SQLDatabaseToolkit(
    db=db, 
    llm=OpenAI(temperature=0)
)

In [4]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [5]:
template = """/
You are a SQL analyst that is querying a database of real estate data. The database has the following tables:

1. **counties**: Stores the identifiers and names of counties.
   - `id`: Unique identifier for each county.
   - `county`: Name of the county.

2. **current_market_stats**: Contains current rental market statistics by zip code.
   - `bedrooms`: Number of bedrooms.
   - `averageRent`: Average rental price.
   - `minRent`: Minimum rental price.
   - `maxRent`: Maximum rental price.
   - `totalListings`: Total number of rental listings.
   - `lastUpdatedDate`: Date when the data was last updated.
   - `zipCode`: Zip code of the rental property.

3. **historic_market_stats**: Holds historical rental market data collected over time by zip code.
   - `bedrooms`: Number of bedrooms.
   - `averageRent`: Historical average rental price.
   - `minRent`: Historical minimum rental price.
   - `maxRent`: Historical maximum rental price.
   - `totalListings`: Historical number of rental listings.
   - `date`: Date of the recorded data.
   - `zipCode`: Zip code of the rental property.

4. **long_term_rentals**: Details long-term rental listings.
   - `property_id`: Unique identifier for the property.
   - `id`: Unique identifier for the listing.
   - `price`: Rental price.
   - `status`: Current status of the listing.
   - `daysOnMarket`: Number of days the property has been on the market.
   - `listedDate`, `createdDate`, `lastSeenDate`, `removedDate`: Relevant dates regarding the property's listing status.

5. **properties**: General information about properties.
   - `property_id`: Unique identifier for the property.
   - `id`: Unique identifier for the record.
   - `formattedAddress`: Full address of the property.
   - `zipCode`: Zip code of the property.
   - `county`: County where the property is located.
   - `subdivision`: Subdivision of the property.
   - `latitude`, `longitude`: Geographical coordinates of the property.
   - `propertyType`: Type of the property (e.g., residential, commercial).
   - `ownerOccupied`: Indicates if the property is owner-occupied.
   - `yearBuilt`: Year the property was built.
   - `lastSaleDate`, `lastSalePrice`: Most recent sale date and price.
   - `zoning`: Zoning classification of the property.
   - `assessorID`, `legalDescription`: Assessor's identification and legal description of the property.

6. **property_features**: Specific features of properties.
   - `property_id`: Unique identifier for the property.
   - `bedrooms`, `bathrooms`: Number of bedrooms and bathrooms.
   - `squareFootage`: Total square footage of the property.
   - `lotSize`: Size of the property's lot.
   - `floorCount`: Number of floors in the property.
   - `garage`, `garageType`: Indicates if there is a garage and its type.
   - `architectureType`, `exteriorType`: Architectural and exterior material type.
   - `heating`, `heatingType`, `cooling`, `coolingType`: Heating and cooling systems and their types.
   - `unitCount`, `garageSpaces`: Number of units and garage spaces.
   - `roofType`, `foundationType`: Type of roof and foundation.
   - `roomCount`: Number of rooms.
   - `fireplace`, `fireplaceType`: Indicates presence of a fireplace and its type.
   - `pool`, `poolType`, `viewType`: Presence of a pool, its type, and view type.

7. **property_owners**: Information about property owners.
   - `owner_id`: Unique identifier for the owner.
   - `property_id`: Unique identifier for the property.
   - `owner`: Name of the property owner.

8. **property_taxes**: Property tax records.
   - `property_tax_id`: Unique identifier for the tax record.
   - `property_id`: Unique identifier for the property.
   - `year`: Tax year.
   - `total`: Total amount of property tax.

9. **property_types**: Descriptions of different property types.
   - `id`: Unique identifier for the property type.
   - `propertyType`: Type of property.
   - `description`: Description of the property type.

10. **sale_listings**: Details about properties listed for sale.
    - `property_id`: Unique identifier for the property.
    - `id`: Unique identifier for the listing.
    - `status`: Current status of the listing (e.g., active, sold).
    - `price`: Listing price.
    - `listedDate`, `removedDate`, `createdDate`, `lastSeenDate`, `daysOnMarket`: Relevant dates and duration on market for the listing.

11. **tax_assessments**: Valuation assessments for tax purposes.
    - `assessment_id`: Unique identifier for the assessment.
    - `property_id`: Unique identifier for the property.
    - `year`: Year of the assessment.
    - `total_value`, `land_value`, `improvements_value`: Total, land, and improvements valuation.

Your job is to write and execute a query that answers the following question:
{query}
"""

prompt = PromptTemplate.from_template(template)

In [7]:
agent_executor.invoke(
    prompt.format(
        query = "What is the address of the property that paid the most property taxes in 2023?"
    )
)



> Entering new SQL Agent Executor chain...
 I need to find the property with the highest property tax payment in 2023.
Action: sql_db_query
Action Input: SELECT formattedAddress FROM properties
            JOIN property_taxes ON properties.property_id = property_taxes.property_id
            WHERE year = 2023
            ORDER BY total DESC
            LIMIT 1[('1215 Silverbell Ter, Charlottesville, VA 22903',)]1215 Silverbell Ter, Charlottesville, VA 22903 is the address of the property with the highest property tax payment in 2023.
Final Answer: 1215 Silverbell Ter, Charlottesville, VA 22903

> Finished chain.


{'input': "/\nYou are a SQL analyst that is querying a database of real estate data. The database has the following tables:\n\n1. **counties**: Stores the identifiers and names of counties.\n   - `id`: Unique identifier for each county.\n   - `county`: Name of the county.\n\n2. **current_market_stats**: Contains current rental market statistics by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Average rental price.\n   - `minRent`: Minimum rental price.\n   - `maxRent`: Maximum rental price.\n   - `totalListings`: Total number of rental listings.\n   - `lastUpdatedDate`: Date when the data was last updated.\n   - `zipCode`: Zip code of the rental property.\n\n3. **historic_market_stats**: Holds historical rental market data collected over time by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Historical average rental price.\n   - `minRent`: Historical minimum rental price.\n   - `maxRent`: Historical maximum rental price.\n   - `totalListing

In [8]:
agent_executor.invoke(
    prompt.format(
        query = "What is the median rent price by zip code?"
    )
)



> Entering new SQL Agent Executor chain...
 I need to find the median rent price for each zip code, so I will need to use the current_market_stats table and group by zip code.
Action: sql_db_query
Action Input: SELECT zipCode, median(averageRent) FROM current_market_stats GROUP BY zipCode[(22901, 1944.0), (22902, 1991.0), (22903, 2463.5), (22911, 2376.0), (22923, 2475.0), (22932, 2600.0), (22936, 1725.0), (22940, 2650.0), (22942, 1462.0), (22947, 3100.0), (22968, 2169.0), (24590, 1562.5)] I need to double check my query before executing it.
Action: sql_db_query_checker
Action Input: SELECT zipCode, median(averageRent) FROM current_market_stats GROUP BY zipCode
SELECT zipCode, median(averageRent) FROM current_market_stats WHERE averageRent IS NOT NULL GROUP BY zipCode I now know the final answer.
Final Answer: The median rent price by zip code is [(22901, 1944.0), (22902, 1991.0), (22903, 2463.5), (22911, 2376.0), (22923, 2475.0), (22932, 2600.0), (22936, 1725.0), (22940, 2650.0), (22

{'input': "/\nYou are a SQL analyst that is querying a database of real estate data. The database has the following tables:\n\n1. **counties**: Stores the identifiers and names of counties.\n   - `id`: Unique identifier for each county.\n   - `county`: Name of the county.\n\n2. **current_market_stats**: Contains current rental market statistics by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Average rental price.\n   - `minRent`: Minimum rental price.\n   - `maxRent`: Maximum rental price.\n   - `totalListings`: Total number of rental listings.\n   - `lastUpdatedDate`: Date when the data was last updated.\n   - `zipCode`: Zip code of the rental property.\n\n3. **historic_market_stats**: Holds historical rental market data collected over time by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Historical average rental price.\n   - `minRent`: Historical minimum rental price.\n   - `maxRent`: Historical maximum rental price.\n   - `totalListing

In [9]:
agent_executor.invoke(
    prompt.format(
        query = "What is the address and square footage of the largest property?"
    )
)



> Entering new SQL Agent Executor chain...
 I need to find the largest property in terms of square footage, so I will need to use the property_features table to get the square footage and the properties table to get the address.
Action: sql_db_query
Action Input: SELECT formattedAddress, squareFootage FROM properties INNER JOIN property_features ON properties.property_id = property_features.property_id ORDER BY squareFootage DESC LIMIT 1[('1540 Avemore Ln, Charlottesville, VA 22911', 336000.0)]I need to make sure my query is correct before executing it.
Action: sql_db_query_checker
Action Input: SELECT formattedAddress, squareFootage FROM properties INNER JOIN property_features ON properties.property_id = property_features.property_id ORDER BY squareFootage DESC LIMIT 1
SELECT formattedAddress, squareFootage 
FROM properties 
INNER JOIN property_features 
ON properties.property_id = property_features.property_id 
WHERE squareFootage IS NOT NULL 
ORDER BY squareFootage DESC 
LIMIT 1; 

{'input': "/\nYou are a SQL analyst that is querying a database of real estate data. The database has the following tables:\n\n1. **counties**: Stores the identifiers and names of counties.\n   - `id`: Unique identifier for each county.\n   - `county`: Name of the county.\n\n2. **current_market_stats**: Contains current rental market statistics by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Average rental price.\n   - `minRent`: Minimum rental price.\n   - `maxRent`: Maximum rental price.\n   - `totalListings`: Total number of rental listings.\n   - `lastUpdatedDate`: Date when the data was last updated.\n   - `zipCode`: Zip code of the rental property.\n\n3. **historic_market_stats**: Holds historical rental market data collected over time by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Historical average rental price.\n   - `minRent`: Historical minimum rental price.\n   - `maxRent`: Historical maximum rental price.\n   - `totalListing

### Test Module

In [1]:
from rentradar.llm.agent import RentRadarLLMAgent

uri = 'duckdb:///../rentradar/db/rentradar.db'

agent = RentRadarLLMAgent(db_uri=uri)

agent.execute_query(query="What is the address of the property that paid the most property taxes in 2023?")

/home/timboslice/.cache/pypoetry/virtualenvs/rentradar-ZiKOXBpo-py3.10/lib/python3.10/site-packages/duckdb_engine/__init__.py:181: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(




> Entering new SQL Agent Executor chain...
 I need to find the property with the highest property tax payment in 2023.
Action: sql_db_query
Action Input: SELECT formattedAddress FROM properties
            JOIN property_taxes ON properties.property_id = property_taxes.property_id
            WHERE year = 2023
            ORDER BY total DESC
            LIMIT 1[('1215 Silverbell Ter, Charlottesville, VA 22903',)]1215 Silverbell Ter, Charlottesville, VA 22903 is the address of the property with the highest property tax payment in 2023.
Final Answer: 1215 Silverbell Ter, Charlottesville, VA 22903

> Finished chain.


{'input': "/\nYou are a SQL analyst that is querying a database of real estate data. The database has the following tables:\n\n1. **counties**: Stores the identifiers and names of counties.\n   - `id`: Unique identifier for each county.\n   - `county`: Name of the county.\n\n2. **current_market_stats**: Contains current rental market statistics by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Average rental price.\n   - `minRent`: Minimum rental price.\n   - `maxRent`: Maximum rental price.\n   - `totalListings`: Total number of rental listings.\n   - `lastUpdatedDate`: Date when the data was last updated.\n   - `zipCode`: Zip code of the rental property.\n\n3. **historic_market_stats**: Holds historical rental market data collected over time by zip code.\n   - `bedrooms`: Number of bedrooms.\n   - `averageRent`: Historical average rental price.\n   - `minRent`: Historical minimum rental price.\n   - `maxRent`: Historical maximum rental price.\n   - `totalListing